# Alchemist: A Python <=> MPI Interface

We start the Alchemist tutorial by playing around with its Python interface. The Python interface itself is running on a single core, but connects to Alchemist, which is running on multiple machines/cores, to do its distributed computations.

Let us use Alchemist to perform some distributed operation on a dataset. In line with what we have seen in the tutorials for the Ristretto package, we will compute the first 200 singular values of a large dataset, the 36GB ocean temperature HDF5 file. In Ristretto we saw how we can do this by loading chunks of the data into memory, in this case we will be sending the data to Alchemist (in chunks), and then we will fetch the results.

## Starting Alchemist

We start by loading some dependencies:

In [2]:
from alchemist import *
import numpy as np

Start an AlchemistSession instance. Hint: AlchemistSession's don't take parameters.

In [3]:
als = AlchemistSession()                  # Start AlchemistSession instance

Connect to Alchemist server at address "0.0.0.0" and port 24960. The Alchemist interface would usually read this information from a file, but for simplicity we input the information directly here. Note that the Alchemist server would normally not be running locally.

In [4]:
address = "0.0.0.0"
port = 0                         # 

als.connect_to_alchemist(address, port)   

Connecting to Alchemist at 0.0.0.0:24960 ...
 
                         Client ID:           0
                         Session ID:          0
                         Command code:        1 (HANDSHAKE)
                         Message body length: 22
                         ----------------------------------------------
                         Datatype (length):   BYTE (1)
                         Data:                4 
 
                         Datatype (length):   SHORT (1)
                         Data:                1234 
 
                         Datatype (length):   STRING (4)
                         Data:                ABCD
 
                         Client ID:           1
                         Session ID:          0
                         Command code:        1 (HANDSHAKE)
                         Message body length: 16
                         ----------------------------------------------
                         Datatype (length):   SHORT (1)
                 

True

Now that we're connected to Alchemist, we need to request some workers. In general, if Alchemist is running on N nodes, then one of those will be the driver node, so there will be N-1 worker nodes. Some of these nodes could be used by other jobs, so we can ask Alchemist how many workers are currently available.

In [5]:
als.list_available_workers()

AttributeError: 'AlchemistSession' object has no attribute 'list_available_workers'

All nodes are currently available, which shouldn't be all too surprising since nobody else is using this particular Alchemist instance. Let's request 2 of the available workers.

In [6]:
num_workers = 1

als.request_workers(num_workers)        # Request 2 workers from Alchemist (if available)

Requesting 2 workers from Alchemist
 
                         Client ID:           1
                         Session ID:          0
                         Command code:        6 (REQUEST_WORKERS)
                         Message body length: 7
                         ----------------------------------------------
                         Datatype (length):   SHORT (1)
                         Data:                2 
 
                         Client ID:           1
                         Session ID:          0
                         Command code:        6 (REQUEST_WORKERS)
                         Message body length: 99
                         ----------------------------------------------
                         Datatype (length):   SHORT (2)
                         Data:                2 1 
 
                         Datatype (length):   STRING (20)
                         Data:                KaisMacBookPro.local
 
                         Datatype (length):   STRING (

To be added

In [7]:
als.workers.connect()

Connecting to Alchemist at 0.0.0.0:24961 ...
 
                         Client ID:           0
                         Session ID:          0
                         Command code:        1 (HANDSHAKE)
                         Message body length: 22
                         ----------------------------------------------
                         Datatype (length):   BYTE (1)
                         Data:                4 
 
                         Datatype (length):   SHORT (1)
                         Data:                1234 
 
                         Datatype (length):   STRING (4)
                         Data:                ABCD
 
                         Client ID:           1
                         Session ID:          1
                         Command code:        1 (HANDSHAKE)
                         Message body length: 16
                         ----------------------------------------------
                         Datatype (length):   SHORT (1)
                 

To be added

In [8]:
als.workers.send_test_string()

Sending test message: 'This is a test message from client 1'
 
                         Client ID:           1
                         Session ID:          1
                         Command code:        4 (SEND_TEST_STRING)
                         Message body length: 41
                         ----------------------------------------------
                         Datatype (length):   STRING (36)
                         Data:                This is a test message from client 1
 
                         Client ID:           1
                         Session ID:          1
                         Command code:        4 (SEND_TEST_STRING)
                         Message body length: 70
                         ----------------------------------------------
                         Datatype (length):   STRING (65)
                         Data:                Alchemist worker received: 'This is a test message from client 1'
Alchemist returned: 'Alchemist worker received: 'This is

## Loading data and sending it to Alchemist

In [10]:
# Read H5 file
f = als.read_from_hdf5("/Users/kai/Downloads/NEON-DS-Imaging-Spectrometer-Data.h5")
# f = als.read_from_hdf5("/mnt/data/sstHD.hdf5")
# Get and print list of datasets within the H5 file
datasetNames = [n for n in f.keys()]
for n in datasetNames:
    print(n)

Reflectance
fwhm
map info
spatialInfo
wavelength


To be added

In [11]:
# sstHD = np.float64(f['Reflectance'][0:10000,:])
sstHD = np.float64(f['Reflectance'][:,:,20])

To be added

In [12]:
alA = als.send_array(sstHD)

426
502
Sending matrix info to Alchemist.
 
                         Client ID:           1
                         Session ID:          0
                         Command code:        16 (MATRIX_INFO)
                         Message body length: 28
                         ----------------------------------------------
                         Datatype (length):   BYTE (2)
                         Data:                0 0 
 
                         Datatype (length):   LONG (2)
                         Data:                426 502 
 
                         Client ID:           1
                         Session ID:          0
                         Command code:        16 (MATRIX_INFO)
                         Message body length: 885
                         ----------------------------------------------
                         Datatype (length):   SHORT (1)
                         Data:                1 
 
                         Datatype (length):   LONG (2)
             

                         Data:                233.0 223.0 302.0 355.0 334.0 360.0 407.0 393.0 362.0 350.0 359.0 463.0 577.0 411.0 417.0 427.0 496.0 465.0 475.0 455.0 451.0 465.0 474.0 494.0 500.0 501.0 499.0 440.0 543.0 575.0 520.0 273.0 230.0 273.0 214.0 317.0 389.0 385.0 444.0 472.0 282.0 175.0 256.0 288.0 276.0 239.0 256.0 328.0 353.0 356.0 310.0 310.0 308.0 435.0 567.0 501.0 460.0 235.0 273.0 317.0 339.0 459.0 440.0 452.0 432.0 270.0 221.0 276.0 408.0 423.0 323.0 321.0 403.0 457.0 483.0 508.0 537.0 603.0 633.0 593.0 339.0 174.0 150.0 151.0 161.0 321.0 225.0 240.0 320.0 295.0 372.0 330.0 267.0 176.0 184.0 358.0 475.0 419.0 482.0 510.0 500.0 482.0 494.0 540.0 561.0 529.0 538.0 436.0 419.0 461.0 555.0 531.0 570.0 672.0 604.0 507.0 518.0 521.0 484.0 503.0 409.0 331.0 331.0 339.0 444.0 455.0 498.0 422.0 349.0 320.0 317.0 288.0 345.0 372.0 408.0 299.0 281.0 360.0 411.0 406.0 385.0 391.0 545.0 489.0 444.0 462.0 454.0 502.0 520.0 581.0 541.0 483.0 354.0 303.0 346.0 359.0 266.0 262.0 311.0 

                         Data:                1878.0 1579.0 1478.0 1352.0 1455.0 1826.0 2545.0 2898.0 2374.0 2583.0 2515.0 3427.0 4354.0 2873.0 2762.0 2996.0 4312.0 4158.0 4200.0 4131.0 4141.0 4056.0 4170.0 4301.0 4394.0 4174.0 4033.0 3090.0 3097.0 3047.0 3065.0 1760.0 2106.0 2318.0 2041.0 2543.0 2697.0 2777.0 2598.0 3011.0 1921.0 1184.0 1595.0 1682.0 1658.0 1506.0 1651.0 2123.0 2088.0 2132.0 2023.0 2109.0 2204.0 3375.0 4707.0 4142.0 3945.0 1723.0 2089.0 2520.0 2740.0 3889.0 3806.0 3937.0 3742.0 2318.0 1946.0 2348.0 3639.0 3714.0 2853.0 2694.0 3703.0 4157.0 4408.0 4548.0 4710.0 4820.0 4728.0 4661.0 2641.0 1123.0 838.0 929.0 939.0 1209.0 778.0 821.0 1072.0 1094.0 1432.0 1338.0 1956.0 1066.0 1292.0 2652.0 3447.0 3037.0 3140.0 3459.0 3801.0 3348.0 3317.0 4176.0 4030.0 3981.0 3540.0 3730.0 4414.0 4217.0 4704.0 4644.0 4659.0 5229.0 5501.0 4988.0 4921.0 4525.0 3809.0 4234.0 3040.0 2313.0 2349.0 2567.0 3565.0 3277.0 3654.0 3297.0 2833.0 2436.0 2210.0 1939.0 2085.0 2184.0 2360.0 2004.0 1801.0 

 
                         Client ID:           1
                         Session ID:          1
                         Command code:        18 (SEND_MATRIX_BLOCKS)
                         Message body length: 864361
                         ----------------------------------------------
                         Datatype (length):   SHORT (1)
                         Data:                1 
 
                         Datatype (length):   LONG (4)
                         Data:                1 1 0 501 
 
                         Datatype (length):   DOUBLE (502)
                         Data:                139.0 178.0 248.0 240.0 236.0 281.0 261.0 272.0 289.0 248.0 286.0 327.0 345.0 306.0 337.0 297.0 296.0 298.0 268.0 316.0 281.0 280.0 300.0 313.0 287.0 274.0 264.0 296.0 395.0 425.0 360.0 228.0 177.0 225.0 177.0 210.0 293.0 277.0 280.0 284.0 201.0 145.0 178.0 216.0 218.0 199.0 197.0 239.0 265.0 268.0 258.0 250.0 260.0 295.0 316.0 250.0 238.0 210.0 211.0 223.0 228.0 272.0 263.0 276

                         Data:                293 293 0 501 
 
                         Datatype (length):   DOUBLE (502)
                         Data:                4326.0 3455.0 3608.0 3701.0 3333.0 3233.0 4295.0 5202.0 4895.0 5546.0 5102.0 6185.0 7103.0 5655.0 5553.0 5945.0 6648.0 6632.0 6987.0 6610.0 6489.0 6573.0 6313.0 7016.0 7079.0 6676.0 6190.0 4859.0 4703.0 4894.0 4658.0 3625.0 4108.0 3864.0 3829.0 4100.0 4374.0 4784.0 4293.0 5029.0 3458.0 2749.0 3238.0 3113.0 2519.0 2781.0 3238.0 3671.0 3788.0 3905.0 3776.0 4038.0 4426.0 4706.0 6568.0 6241.0 6117.0 4129.0 4317.0 4415.0 4393.0 5823.0 6362.0 5600.0 5418.0 3754.0 3557.0 4056.0 5014.0 5383.0 5046.0 4834.0 5610.0 6478.0 6137.0 6797.0 7402.0 7198.0 6899.0 6631.0 4640.0 2724.0 2747.0 2778.0 2588.0 3636.0 2866.0 2773.0 2859.0 3203.0 3735.0 3218.0 4624.0 2893.0 3337.0 5242.0 6509.0 5518.0 5223.0 6201.0 6989.0 6307.0 5846.0 7314.0 7984.0 8088.0 7130.0 6964.0 7569.0 8125.0 8807.0 8968.0 8809.0 8795.0 10745.0 9287.0 9090.0 8288.0 7182.

## Loading a library

In [13]:
testlib = als.load_library("TestLib")  

 
                         Client ID:           1
                         Session ID:          0
                         Command code:        13 (LOAD_LIBRARY)
                         Message body length: 61
                         ----------------------------------------------
                         Datatype (length):   STRING (7)
                         Data:                TestLib
 
                         Datatype (length):   STRING (44)
                         Data:                /Users/kai/Projects/AlLib/target/allib.dylib
 
                         Client ID:           1
                         Session ID:          0
                         Command code:        13 (LOAD_LIBRARY)
                         Message body length: 7
                         ----------------------------------------------
                         Datatype (length):   SHORT (1)
                         Data:                1 


## Running a task on Alchemist

To be added

In [14]:
alU, alS, alV = als.run_task("truncated_SVD", alA, 20)

 
                         Client ID:           1
                         Session ID:          0
                         Command code:        14 (RUN_TASK)
                         Message body length: 40
                         ----------------------------------------------
                         Datatype (length):   STRING (13)
                         Data:                truncated_SVD
 
                         Datatype (length):   SHORT (1)
                         Data:                1 
 
                         Datatype (length):   INT (1)
                         Data:                20 
 
                         Datatype (length):   BYTE (1)
                         Data:                0 
 
                         Client ID:           1
                         Session ID:          0
                         Command code:        14 (RUN_TASK)
                         Message body length: 1985
                         ----------------------------------------------
   

To be added

In [15]:
S = als.get_array(alS)

print(S)

 
                         Client ID:           1
                         Session ID:          1
                         Command code:        19 (REQUEST_MATRIX_BLOCKS)
                         Message body length: 332
                         ----------------------------------------------
                         Datatype (length):   SHORT (1)
                         Data:                3 
 
                         Datatype (length):   LONG (40)
                         Data:                0 0 0 0 2 2 0 0 4 4 0 0 6 6 0 0 8 8 0 0 10 10 0 0 12 12 0 0 14 14 0 0 16 16 0 0 18 18 0 0 
 
                         Client ID:           1
                         Session ID:          3
                         Command code:        19 (REQUEST_MATRIX_BLOCKS)
                         Message body length: 507
                         ----------------------------------------------
                         Datatype (length):   SHORT (1)
                         Data:                3 
 
       

## Stopping Alchemist

Once done with Alchemist, it is important to stop the AlchemistSession instance using the stop() command. This frees up resources that Alchemist can then allocate to other jobs.

In [ ]:
als.stop()